# 🔓 RSA Attack Level 3 – Reuse of Public Key with Small e

Serangan ini terjadi ketika **pesan yang sama** dienkripsi ke beberapa penerima berbeda menggunakan eksponen kecil (e = 3). Jika `m^e < n1*n2*n3`, attacker bisa menggabungkan ciphertext dan mengambil akar ke-e untuk mendapatkan plaintext **tanpa private key**.

📌 Teknik: Håstad’s Broadcast Attack + Chinese Remainder Theorem

✅ Syarat:
- e = 3
- 3 modulus berbeda (coprime)
- Pesan yang sama (tanpa padding)

🎯 Tujuan: Mendapatkan plaintext dari ciphertext gabungan.


In [11]:
from Crypto.Util.number import getPrime, bytes_to_long, long_to_bytes  # Library untuk bilangan prima dan konversi teks-angka
from functools import reduce                                           # Untuk perkalian semua elemen list (CRT)
import gmpy2                                                           # Library matematika cepat (akar, mod inverse, dsb.)

# Fungsi Chinese Remainder Theorem (gabungkan beberapa ciphertext dengan modulus berbeda)
def crt(c_list, n_list):
    N = reduce(lambda a, b: a * b, n_list)  # N adalah hasil kali semua modulus
    total = 0                               # Inisialisasi hasil
    for c_i, n_i in zip(c_list, n_list):    # Iterasi setiap pasangan ciphertext dan modulus
        m_i = N // n_i                      # Hitung bagian modulus lain
        inv = gmpy2.invert(m_i, n_i)        # Hitung invers modulo
        total += c_i * m_i * inv            # Tambahkan kontribusi dari pasangan tersebut
    return total % N                        # Modulo dengan N untuk hasil akhir CRT


In [12]:
# Plaintext yang akan dienkripsi (bisa disesuaikan)
plaintext = b"Daffa Dimas Nova"              # Plaintext dalam bentuk byte
m = bytes_to_long(plaintext)                 # Konversi plaintext menjadi integer
e = 3                                        # Eksponen publik kecil (vulnerable jika reuse)

# Fungsi untuk membuat modulus n = p × q dengan panjang 512-bit
def generate_modulus(bits=512):
    return getPrime(bits) * getPrime(bits)   # Ambil dua bilangan prima acak dan kalikan



In [13]:
n_list = [generate_modulus() for _ in range(3)]  # Buat 3 modulus RSA berbeda
c_list = [pow(m, e, n) for n in n_list]          # Enkripsi pesan yang sama dengan e yang sama

c_combined = crt(c_list, n_list)                 # Gabungkan ciphertext dengan CRT

# Ambil akar pangkat 3 dari hasil CRT → tujuannya untuk membalik m^e menjadi m
recovered_m, exact = gmpy2.iroot(c_combined, e)  # iroot mengembalikan akar dan flag "exact"
recovered_text = long_to_bytes(recovered_m)      # Konversi integer kembali ke bentuk teks


In [14]:
print("===== Attack 3: Reuse Public Key with Small e (gmpy2) =====")
for i in range(3):
    print(f"Modulus n{i+1} = {n_list[i]}")          # Cetak setiap modulus
    print(f"Ciphertext c{i+1} = {c_list[i]}")       # Cetak setiap ciphertext

print("\nHasil gabungan CRT:", c_combined)          # Cetak hasil CRT
print(f"Akar pangkat 3 (recovered m): {recovered_m}") # Cetak integer hasil dekripsi
print(f"Teks hasil dekripsi: '{recovered_text.decode()}'")  # Konversi ke string dan tampilkan
print(f"Berhasil? {recovered_text == plaintext}")    # Verifikasi apakah hasil sama dengan plaintext awal


===== Attack 3: Reuse Public Key with Small e (gmpy2) =====
Modulus n1 = 94448915305838991146332311792037526830549535233715395810857276104996565008448574863188786028170501595796123811061666398492475965379294999501459885488435873638503980525968837859509693004918068293595369754015980001892653343797527441404366933174918137830913329778012402217435734132935864876973569145327076985231
Ciphertext c1 = 750921708733871343535259110877586015283603368964208968626728870807915851311530807651019564503018616501443749334817
Modulus n2 = 171085485453507842172130331277917009885556241292156675457013854785365102914764329647340473881314931572823721920085165530443986212779623136631226231755148370822191317081191606742964222305846276585891529677017760549457820925397859873323799823645948296657206076774127476005386040312581164886959895546841770997617
Ciphertext c2 = 750921708733871343535259110877586015283603368964208968626728870807915851311530807651019564503018616501443749334817
Modulus n3 = 97008429514561009889